## [Chowdhary et al. 2020] data comparison
This script compares measured Stokes vectors above an atmosphere-ocean system described in [Chowdhary et al. 2020].

In [1]:
import sys
sys.path.append('..')

import math
import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, interact

from parse_egap_data import get_data

In [2]:
wavelength_values = [350., 450., 550., 650.]
theta_photon_values = [30.0, 60.0]
phi_values = [0.0, 60.0, 180.0, 240.0]
theta_sensor_values = np.linspace(0.0, 60.0, 13)

In [3]:
def selection_slider(options, value, description):
    return widgets.SelectionSlider(options = options, value = value,description = description, 
                                   continuous_update = False, orientation = 'horizontal', readout = True, 
                                   layout = widgets.Layout(width = '95%'))

model_select = widgets.ToggleButtons(options = ['AOS_1', 'AOS_2', 'AOS_3'], description='model', disabled=False) 

stokes_select = widgets.ToggleButtons(options = ['R_I', 'DOP'], description='data', disabled=False,
    tooltips=['Total reflectance', 'Degree of polarization'],
)

wav_select = selection_slider(options = wavelength_values, value = 350.0, description = 'wavelength')
theta_photon_select = selection_slider(options = theta_photon_values, value = 30.0, description = 'theta_photon')
phi_select = selection_slider(options = phi_values, value = 0.0, description = 'phi')

ui_elements = [model_select, stokes_select, wav_select, theta_photon_select, phi_select]
ui = widgets.VBox(children = ui_elements)

plot_ref = go.Scatter(x = np.array(theta_sensor_values), y = np.zeros(len(theta_sensor_values)),
                      mode = 'lines', 
                      line = dict(), 
                      name = '[Chowdhary et al. 2020]')

plot_mts = go.Scatter(x = np.array(theta_sensor_values), y = np.zeros(len(theta_sensor_values)),
                      mode = 'lines', 
                      line = dict(), 
                      name = 'Ours')

plot_diff = go.Scatter(x = np.array(theta_sensor_values), y = np.zeros(len(theta_sensor_values)),
                      mode = 'lines', 
                      line = dict(), 
                      name = '% Error',
                      yaxis = 'y2')

layout = go.Layout(
    xaxis_title_text = 'theta_sensor',
    yaxis = dict(
                title_text = 'Reflectance',
                range = [0.0, 1.0]
            ),
    yaxis2 = dict(
                title_text = '% Error',
                range = [-10.0, 10.0],
                anchor = 'x',
                overlaying = 'y',
                side = 'right',
                showgrid = False
            ),
    showlegend = True
)

fig = go.FigureWidget(data = [plot_ref, plot_mts, plot_diff], layout = layout)

def update(model_select, stokes_select, wav_select, theta_photon_select, phi_select):
    
    # Load experiment data
    mts_file = '../' + model_select + '_mts.json'

    with open(mts_file, 'r') as f:
        data_mts = json.load(f)

    mu_photon = math.cos(np.deg2rad(theta_photon_select))

    if stokes_select == 'R_I':
        data_ref_new = get_data('../egap_data', model_select, theta_photon_select, phi_select, wav_select)
        data_ref_new = data_ref_new[:,1]
        
        data_mts_tmp = data_mts[stokes_select[2]][str(wav_select)][str(theta_photon_select)]
        data_mts_new = [data_mts_tmp[str(s)][str(phi_select)] for s in theta_sensor_values]
        data_mts_new = np.array(data_mts_new)

        # Convert to reflectance quantity
        data_mts_new = data_mts_new / mu_photon

    elif stokes_select == 'DOP':
        data_ref_new = get_data('../egap_data', model_select, theta_photon_select, phi_select, wav_select)
        
        data_ref_RI = data_ref_new[:,1]
        data_ref_RQ = data_ref_new[:,2]
        data_ref_RU = data_ref_new[:,3]
        
        # Convert to original Stokes vector quantity
        data_ref_I  = data_ref_RI * mu_photon
        data_ref_Q  = data_ref_RQ * mu_photon
        data_ref_U  = data_ref_RU * mu_photon
        
        data_ref_new = np.sqrt(data_ref_Q * data_ref_Q + data_ref_U * data_ref_U) / data_ref_I * 100.0
        
        data_mts_tmp = data_mts['I'][str(wav_select)][str(theta_photon_select)]
        data_mts_I = np.array([data_mts_tmp[str(s)][str(phi_select)] for s in theta_sensor_values])
        data_mts_tmp = data_mts['Q'][str(wav_select)][str(theta_photon_select)]
        data_mts_Q = np.array([data_mts_tmp[str(s)][str(phi_select)] for s in theta_sensor_values])
        data_mts_tmp = data_mts['U'][str(wav_select)][str(theta_photon_select)]
        data_mts_U = np.array([data_mts_tmp[str(s)][str(phi_select)] for s in theta_sensor_values])
        
        data_mts_new = np.sqrt(data_mts_Q * data_mts_Q + data_mts_U * data_mts_U) / data_mts_I * 100.0
        
    else:
        pass
    
    # Update plots
    with fig.batch_update():
        fig.data[0].y = data_ref_new
        fig.data[1].y = data_mts_new
        fig.data[2].y = (data_mts_new - data_ref_new) / data_ref_new * 100.0
        
        if stokes_select == 'DOP':
            fig.layout.yaxis.range = [0.0, 100.0]
            fig.layout.yaxis.title.text = 'DOP'
        else:
            dmax = np.amax(data_ref_new)
            fig.layout.yaxis.range = [0.0, 1.0] if dmax <= 1.0 else [0.0, dmax]
            fig.layout.yaxis.title.text = 'Reflectance'

out = widgets.interactive_output(update, {'model_select': model_select, 'stokes_select': stokes_select, 
                                          'wav_select': wav_select, 'theta_photon_select': theta_photon_select, 
                                          'phi_select': phi_select})

display(ui)
fig

FigureWidget({
    'data': [{'mode': 'lines',
              'name': '[Chowdhary et al. 2020]',
              '…